In [5]:
using LinearAlgebra
using FFTW
using Statistics
using JLD2
using NPZ
using ProgressMeter

In [14]:
using BenchmarkTools

next_fast_len (generic function with 1 method)

In [2]:
using NPZ

outputpath = "/media/dan/Data/git/network_mining/connectivity/julia_test/"
data = npzread("/media/dan/Data/git/network_mining/connectivity/julia_test/034_input.npy")
sfreq = 2048
freqs = collect(14:1023)
zero_mean = true
n_freqs = length(freqs)
mt_bandwidth = 4
n_taps = floor(Int, mt_bandwidth - 1)
n_cycles = 7
n_epochs, n_channels, n_times = size(data)

(606, 97, 1024)

In [ ]:
typeof(data)

Array{Float64, 3}

In [12]:
Ws, weights = compute_tapers(n_times, n_taps, freqs, mt_bandwidth, n_cycles, sfreq)

(Vector{ComplexF64}[[-0.001214274983100297 + 3.0887961663277934e-17im, -0.0012520856711446579 - 5.215534270503756e-5im, -0.0012881041434226464 - 0.00010760543953783077im, -0.0013221174190464562 - 0.00016631518578612118im, -0.001353911213620343 - 0.00022823736645790528im, -0.0013832705778518367 - 0.00029331243311249655im, -0.001409980555104932 - 0.0003614683126338824im, -0.0014338268563638084 - 0.00043262024926574253im, -0.0014545965509924234 - 0.0005066706811567809im, -0.0014720787715949309 - 0.0005835091525895943im  …  -0.0014860654312048316 + 0.0006630122629856336im, -0.001472078771594719 + 0.0005835091525895718im, -0.0014545965509922102 + 0.0005066706811567685im, -0.0014338268563635988 + 0.0004326202492657413im, -0.001409980555104721 + 0.00036146831263389066im, -0.0013832705778516314 + 0.00029331243311251547im, -0.0013539112136201415 + 0.0002282373664579341im, -0.0013221174190462606 + 0.00016631518578615962im, -0.0012881041434224547 + 0.00010760543953787803im, -0.001252085671144473 

In [ ]:
function _get_nfft(Ws::Matrix{Vector{ComplexF64}}, X::Array{Float64, 3})::Int
    max_len = maximum([length(Wk) for Wk in Ws])
    n = last(size(X))
    nfft = n + max_len - 1
    # @show nfft
    nfft = next_fast_len(nfft)
    return nfft
end

function zero_pad!(data_padded::Vector{ComplexF64}, data::Vector{ComplexF64})::Vector{ComplexF64}
    data_padded[1:length(data)] .= data
    data_padded[length(data)+1:end] .= 0
end

function precompute_fft_Ws(Ws, nfft)
    n_taps, n_freqs = size(Ws)
    fft_Ws = Array{ComplexF64,3}(undef, n_taps, n_freqs, nfft)
    for taper_idx = 1:n_taps
        for freq_idx = 1:n_freqs
            W = Ws[taper_idx, freq_idx]
            padded_W = zeros(ComplexF64, nfft)
            zero_pad!(padded_W, W)
            fft_Ws[taper_idx, freq_idx, :] = fft(padded_W)
        end
    end
    return fft_Ws
end

In [29]:
begin
    @benchmark _get_nfft(Ws, data);
end

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.163 μs …  2.605 ms  ┊ GC (min … max): 0.00% … 85.56%
 Time  (median):     4.858 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.344 μs ± 31.915 μs  ┊ GC (mean ± σ):  7.16% ±  1.70%

   ▅██▇▆▅▃▂                                           ▁▂▃▃▂▁ ▂
  ███████████▆▇▇▅▆▁▅▄▅▁▁▁▃▁▄▅▄▅▅▄▄▆▃▄▅▅▆▄▃▄▁▁▁▁▁▁▃▁▁▁▇██████ █
  4.16 μs      Histogram: log(frequency) by time     15.2 μs <

 Memory estimate: 25.25 KiB, allocs estimate: 5.

In [28]:
begin
    @benchmark precompute_fft_Ws(Ws, _get_nfft(Ws, data));
end

LoadError: InterruptException: